In [2]:
!pip install geopandas
!pip install backoff
!pip install statsmodels
!pip install seaborn
!pip install scipy
!pip install mapclassify
!pip install unidecode
!pip install rasterio
!pip install seaborn
!pip install mpl_scatter_density
!pip install rasterstats
!pip install xmltodict
!pip install pandas

  Using cached fiona-1.9.5-cp310-cp310-manylinux2014_x86_64.whl.metadata (49 kB)
  Using cached pandas-2.1.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached pyproj-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
  Using cached shapely-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.9 MB/s eta 0:00:00:00:010:01
Using cached fiona-1.9.5-cp310-cp310-manylinux2014_x86_64.whl (15.7 MB)
Using cached pandas-2.1.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
Using cached pyproj-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)
Using cached shapely-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86

In [3]:
import os
import subprocess
import geopandas
import pandas as pd
import json
import datetime
import backoff
import xmltodict
import fileinput
import xml.etree.ElementTree as ET
from urllib.parse import urlparse
import fileinput
import glob
import matplotlib.pyplot as plt
# import numpy as np
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import scipy.stats
import scipy.stats as stats
from statsmodels import graphics
import shutil
from mpl_toolkits.axes_grid1 import make_axes_locatable
from unidecode import unidecode
import rasterio
from rasterio.plot import show
import seaborn
# import mpl_scatter_density # adds projection='scatter_density'
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors
from textwrap import wrap
import matplotlib.patches as mpatches
import pylab as py
import rasterstats
from rasterstats import zonal_stats
import scipy
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas as pd
import subprocess
import glob
import shutil
import time

from maap.maap import MAAP
# maap=MAAP(maap_host="api.ops.maap-project.org") 
maap=MAAP() 

def BUILD_TABLE_JOBSTATUS(submit_results_df, status_col = 'status'):
    import xmltodict
    
    # If jobs failed to submit, then they have a NaN for jobid, which makes the merge (join) fail
    submit_results_df = submit_results_df.fillna('')
    
    job_status_df = pd.concat([pd.DataFrame({'job_id': [job_id], 'status':[maap.getJobStatus(job_id)]}) for job_id in submit_results_df.job_id.to_list()])
    job_status_df = submit_results_df.merge(job_status_df, how='left', left_on='job_id',  right_on='job_id')
    
    print(f'Count total jobs:\t{len(job_status_df)}')
    print(f"Count pending jobs:\t{job_status_df[job_status_df[status_col] =='Accepted'].shape[0]}")
    print(f"Count running jobs:\t{job_status_df[job_status_df[status_col] =='Running'].shape[0]}")
    
    NUM_FAILS = job_status_df[job_status_df[status_col] =='Failed'].shape[0]
    NUM_SUCCEEDS = job_status_df[job_status_df[status_col] =='Succeeded'].shape[0]
    print(f"Count succeeded jobs:\t{NUM_SUCCEEDS}")
    print(f"Count failed jobs:\t{NUM_FAILS}")
    if NUM_FAILS > 0:
        print(f"% of failed jobs:\t{round(NUM_FAILS / ( NUM_FAILS + NUM_SUCCEEDS ), 4) * 100}\n")
    else:
        print(f"% of failed jobs:\tNothing has failed...yet\n")
    
    return job_status_df

### DPS Jobs for all TILES

#### COMMIT ALL CHANGES and REGISTER ALGORITHM FROM TERMINAL

In [3]:
# cd /projects/ADE_biomass_harmonization/NASA_CMS
# git add NASA_CMS_2023/Hunka_2024_NSD/Forest_classification
# git commit -m ' Classifying the world into IPCC Table 4.7'
# git tag -f IPCC_classes_ADE_v8

# git push origin -f IPCC_classes_ADE_v8
# git push
# git push origin -f IPCC_classes_ADE_v8

In [6]:
########### REGISTER ALGORITHM ON NEW ADE ##############
maap.register_algorithm_from_yaml_file("/projects/ADE_biomass_harmonization/NASA_CMS/NASA_CMS_2023/Hunka_2024_NSD/Forest_classification/run_IPCC_classes_ADE.yml").text

'{"code": 200, "message": {"id": "baeaeb8a9b36bfc85b45d2eb1fc7e3b55b189345", "short_id": "baeaeb8a", "created_at": "2023-12-13T04:00:03.000+00:00", "parent_ids": ["3ab81e04b3b861b3d84f5a00e867cc0dce9e2e79"], "title": "Registering algorithm: run_IPCC_classes_ADE", "message": "Registering algorithm: run_IPCC_classes_ADE", "author_name": "root", "author_email": "root@e49d5cea4b76", "authored_date": "2023-12-13T04:00:03.000+00:00", "committer_name": "root", "committer_email": "root@e49d5cea4b76", "committed_date": "2023-12-13T04:00:03.000+00:00", "trailers": {}, "web_url": "https://repo.maap-project.org/root/register-job-hysds-v4/-/commit/baeaeb8a9b36bfc85b45d2eb1fc7e3b55b189345", "stats": {"additions": 0, "deletions": 0, "total": 0}, "status": "pending", "project_id": 3, "last_pipeline": {"id": 10524, "iid": 650, "project_id": 3, "sha": "baeaeb8a9b36bfc85b45d2eb1fc7e3b55b189345", "ref": "main", "status": "pending", "source": "push", "created_at": "2023-12-13T04:00:04.467Z", "updated_at": 

#### RUN ALGORITHM FROM TERMINAL / SUBMIT JOBS TO DPS FOR ALL TILES ####

In [9]:
Desired_output_folder = '/projects/my-public-bucket/Data/Harris_et_al_PAPER/Global_IPCC_classes_ADE_v8_COG/'
if not os.path.exists(Desired_output_folder):
    os.mkdir(Desired_output_folder)
TILES = ["00N_000E","00N_010E","00N_020E","00N_030E","00N_040E","00N_040W","00N_050W","00N_060W","00N_070W","00N_080W","00N_090E","00N_090W","00N_100E","00N_110E","00N_120E","00N_130E","00N_140E","00N_150E","00N_160E","10N_000E","10N_010E","10N_010W","10N_020E","10N_020W","10N_030E","10N_040E","10N_050E","10N_050W","10N_060W","10N_070E","10N_070W","10N_080E","10N_080W","10N_090E","10N_090W","10N_100E","10N_110E","10N_120E","10N_130E","10S_010E","10S_020E","10S_030E","10S_040E","10S_040W","10S_050E","10S_050W","10S_060W","10S_070W","10S_080W","10S_110E","10S_120E","10S_130E","10S_140E","10S_150E","10S_160E","10S_170E","20N_000E","20N_010E","20N_010W","20N_020E","20N_020W","20N_030E","20N_040E","20N_050E","20N_060W","20N_070E","20N_070W","20N_080E","20N_080W","20N_090E","20N_090W","20N_100E","20N_100W","20N_110E","20N_110W","20N_120E","20N_160W","20S_010E","20S_020E","20S_030E","20S_040E","20S_050W","20S_060W","20S_070W","20S_080W","20S_110E","20S_120E","20S_130E","20S_140E","20S_150E","20S_160E","30N_000E","30N_010E","30N_010W","30N_020E","30N_020W","30N_030E","30N_040E","30N_050E","30N_060E","30N_070E","30N_080E","30N_080W","30N_090E","30N_090W","30N_100E","30N_100W","30N_110E","30N_110W","30N_120E","30N_120W","30N_130E","30N_160W","30N_170W","30S_010E","30S_020E","30S_030E","30S_060W","30S_070W","30S_080W","30S_110E","30S_120E","30S_130E","30S_140E","30S_150E","30S_170E","40N_000E","40N_010E","40N_010W","40N_020E","40N_020W","40N_030E","40N_040E","40N_050E","40N_060E","40N_070E","40N_080E","40N_080W","40N_090E","40N_090W","40N_100E","40N_100W","40N_110E","40N_110W","40N_120E","40N_120W","40N_130E","40N_130W","40N_140E","40S_070W","40S_080W","40S_140E","40S_160E","40S_170E","50N_000E","50N_010E","50N_010W","50N_020E","50N_030E","50N_040E","50N_050E","50N_060E","50N_060W","50N_070E","50N_070W","50N_080E","50N_080W","50N_090E","50N_090W","50N_100E","50N_100W","50N_110E","50N_110W","50N_120E","50N_120W","50N_130E","50N_130W","50N_140E","50N_150E","50S_070W","50S_080W","60N_000E","60N_010E","60N_010W","60N_020E","60N_020W","60N_030E","60N_040E","60N_050E","60N_060E","60N_060W","60N_070E","60N_070W","60N_080E","60N_080W","60N_090E","60N_090W","60N_100E","60N_100W","60N_110E","60N_110W","60N_120E","60N_120W","60N_130E","60N_130W","60N_140E","60N_140W","60N_150E","60N_150W","60N_160E","60N_160W","60N_170E","60N_170W","60N_180W","70N_000E","70N_010E","70N_020E","70N_030E","70N_040E","70N_050E","70N_060E","70N_070E","70N_070W","70N_080E","70N_080W","70N_090E","70N_090W","70N_100E","70N_100W","70N_110E","70N_110W","70N_120E","70N_120W","70N_130E","70N_130W","70N_140E","70N_140W","70N_150E","70N_150W","70N_160E","70N_160W","70N_170E","70N_170W","80N_010E","80N_020E","80N_030E","80N_070E","80N_080E","80N_090E","80N_100E","80N_110E","80N_120E","80N_130E","80N_130W","80N_140E","80N_140W","80N_150E","80N_150W","80N_160E","80N_160W","80N_170W"]

i = 0
count = -1
submit_results_df_list = []
len_input_list = len(TILES)
list_jobIDs = [""]*len(TILES)

for TILE in TILES: 
    if not os.path.exists(os.path.join(Desired_output_folder,'IPCC_classes/' + TILE + "_IPCC_classes.tif")): 
        print("submitting " + TILE)
        count +=1
        submit_result = maap.submitJob(identifier="IPCC_classes_ADE", 
                       algo_id="run_IPCC_classes_ADE",
                       version="IPCC_classes_ADE_v8",
                       username="nehajo88",
                       queue="maap-dps-worker-32vcpu-64gb",#"maap-dps-worker-32gb",maap-dps-worker-32vcpu-64gb
                       FOREST_HEIGHT_2000="https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_2000/2000_" + TILE + ".tif",
                       FOREST_HEIGHT_2020="https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_2020/2020_" + TILE + ".tif",
                       FOREST_LOSS="https://glad.umd.edu/users/Potapov/GLCLUC2020/Forest_height_loss_2000_2020/netloss_" + TILE + ".tif",
                       ESA_FOREST_COVER="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/ESA_WorldCover2021_v2/WC_"+ TILE + "_C.tif",
                       FOREST_COVER_LOSSYEAR="https://storage.googleapis.com/earthenginepartners-hansen/GFC-2022-v1.10/Hansen_GFC-2022-v1.10_lossyear_"+ TILE + ".tif",
                       JRC_TMST="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/JRC_TFM/"+TILE+"_JRC_Transition_Map.tif",
                       JRC_DEF="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/JRC_DEF/"+TILE+"_JRC_Deforestation_Map.tif",
                       JRC_DEG="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/JRC_DEG/"+TILE+"_JRC_Degradation_Map.tif",
                       BOREAL_AGE="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/Forest_age_boreal/BOREAL_AGE/"+ TILE + "_BOREAL_AGE.tif",
                       PLANTATIONS="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/WRI_Planted_Forest_Type_v2/"+ TILE + "_plantation_type_oilpalm_woodfiber_other.tif",
                       GFM="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/IIASA/"+ TILE + "_GFM.tif",
                       IFL_rasters="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/Intact_Forest_Landscapes/IFL_rasters_2020/"+ TILE + "_IFL_2020.tif",
                       FII_raster="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/Forest_Integrity_Index/"+ TILE + "_FII.tif",                
                       primary_forest_asia="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/Turubanova_Humid_tropical_primary_forest/"+ TILE + "_Asia.tif",
                       primary_forest_sa="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/Turubanova_Humid_tropical_primary_forest/"+ TILE + "_SouthAmerica.tif",
                       primary_forest_africa="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/Turubanova_Humid_tropical_primary_forest/"+ TILE + "_Africa.tif",
                       EcoCont="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/Harris_et_al_PAPER/EcoCont_v2/"+ TILE + "_EcoCont.tif",
                       tile= TILE,
                       output_mask= TILE + "_Age_classes.tif",
                       output_file= TILE + "_IPCC_classes.tif")
        job_id = str(submit_result).split(': ')[1].split(',')[0].replace("'","")
        print(job_id)
        time.sleep(1)

### ASSESS DPS RESULTS 

In [8]:
Desired_output_folder = '/projects/my-public-bucket/Data/Harris_et_al_PAPER/Global_IPCC_classes_ADE_v8_COG/IPCC_classes/'
if not os.path.exists(Desired_output_folder):
    os.mkdir(Desired_output_folder)
SUMMARY_files_tifs = glob.glob('/projects/my-private-bucket/dps_output/run_IPCC_classes_ADE/IPCC_classes_ADE_v8/IPCC_classes_ADE/2024/**/*IPCC*.tif', recursive=True)
print("Moving files...")
for each_file in SUMMARY_files_tifs:
    if os.path.exists(each_file) and not os.path.exists(os.path.join(Desired_output_folder,os.path.basename(each_file))): 
        print(each_file)
        shutil.move(each_file, Desired_output_folder)

Desired_output_folder = '/projects/my-public-bucket/Data/Harris_et_al_PAPER/Global_IPCC_classes_ADE_v8_COG/Age_classes/'
if not os.path.exists(Desired_output_folder):
    os.mkdir(Desired_output_folder)
SUMMARY_files_tifs = glob.glob('/projects/my-private-bucket/dps_output/run_IPCC_classes_ADE/IPCC_classes_ADE_v8/IPCC_classes_ADE/2024/**/*Age*.tif', recursive=True)
print("Moving files...")
for each_file in SUMMARY_files_tifs:
    if os.path.exists(each_file) and not os.path.exists(os.path.join(Desired_output_folder,os.path.basename(each_file))): 
        print(each_file)
        shutil.move(each_file, Desired_output_folder)

Moving files...
/projects/my-private-bucket/dps_output/run_IPCC_classes_ADE/IPCC_classes_ADE_v7/IPCC_classes_ADE/2024/01/08/10/09/44/367491/30N_090E_IPCC_classes.tif
/projects/my-private-bucket/dps_output/run_IPCC_classes_ADE/IPCC_classes_ADE_v7/IPCC_classes_ADE/2024/01/08/10/09/48/815918/40S_080W_IPCC_classes.tif
/projects/my-private-bucket/dps_output/run_IPCC_classes_ADE/IPCC_classes_ADE_v7/IPCC_classes_ADE/2024/01/08/10/10/03/161849/50N_050E_IPCC_classes.tif
/projects/my-private-bucket/dps_output/run_IPCC_classes_ADE/IPCC_classes_ADE_v7/IPCC_classes_ADE/2024/01/08/10/10/06/895946/50N_060E_IPCC_classes.tif
/projects/my-private-bucket/dps_output/run_IPCC_classes_ADE/IPCC_classes_ADE_v7/IPCC_classes_ADE/2024/01/08/10/10/34/130405/50N_010W_IPCC_classes.tif
/projects/my-private-bucket/dps_output/run_IPCC_classes_ADE/IPCC_classes_ADE_v7/IPCC_classes_ADE/2024/01/08/10/10/36/555674/50N_070E_IPCC_classes.tif
/projects/my-private-bucket/dps_output/run_IPCC_classes_ADE/IPCC_classes_ADE_v7/IPCC

### ZIP TO SHARE - RUN COMMANDS ON TERMINAL

In [ ]:
zip -0 /projects/my-public-bucket/Data/Harris_et_al_PAPER/Global_IPCC_classes_ADE_v8/IPCC_classes_v6.zip *IPCC*
zip -0 /projects/my-public-bucket/Data/Harris_et_al_PAPER/Global_IPCC_classes_ADE_v8/Age_classes_v6.zip *Age*
zip -0 /projects/my-public-bucket/Data/Harris_et_al_PAPER/Global_IPCC_classes_ADE_v8_GPKG/IPCC_classes_v6_GPKG.zip *gpkg*